In [39]:
import sys
for p in ['.', '..']:
  if p not in sys.path:
    sys.path.insert(0, p)

from collections import namedtuple
import os
import pickle

import numpy as np
import scipy.io as spio

from digits.data import *

In [41]:
loader = Loader('..')
loader.assert_ready()
loader.clear_pickled()
d = loader.read_data('cropped', 'test')